# ETL Processes
Use this notebook to develop the ETL process for each of your tables before completing the `etl.py` file to load the whole datasets.

### Data Sources
1. http://millionsongdataset.com/
1. https://github.com/Interana/eventsim

In [262]:
import os
import glob
import psycopg2
import pandas as pd
from sql_queries import *

In [263]:
conn = psycopg2.connect("host=127.0.0.1 dbname=sparkifydb user=student password=student")
cur = conn.cursor()

In [264]:
def get_files(filepath):
    all_files = []
    for root, dirs, files in os.walk(filepath):
        files = glob.glob(os.path.join(root,'*.json'))
        for f in files :
            all_files.append(os.path.abspath(f))
    
    return all_files

### Breaking down get_files

os.walk will create a list of tuples for each level in a folder subdirectory. Each tuple has:
1. path to currently referenced directory
2. a list of folders in this directory (if any)
3. a list of files in this direction (if any)

the 2nd or 3rd tuple will often be empty depending on the structure of the folders. 


glob.glob will list out all the files and folders matching a search string, usually using the *..similar to ls grep. We are doing this to make sure to only grab the json files in case something else gets thrown in there

finally, I'm not sure why abs path is necessary as walk should only return the absolute filepath, but perhaps its to prevent weird edge cases

# Process `song_data`
In this first part, you'll perform ETL on the first dataset, `song_data`, to create the `songs` and `artists` dimensional tables.

Let's perform ETL on a single song file and load a single record into each table to start.
- Use the `get_files` function provided above to get a list of all song JSON files in `data/song_data`
- Select the first song in this list
- Read the song file and view the data

In [265]:
song_files = get_files('data/song_data/')

In [266]:
# List out the song files
[i.split('/')[-1] for i in song_files]

['TRABBKX128F4285205.json',
 'TRABBOR128F4286200.json',
 'TRABBNP128F932546F.json',
 'TRABBAM128F429D223.json',
 'TRABBJE12903CDB442.json',
 'TRABBTA128F933D304.json',
 'TRABBVJ128F92F7EAA.json',
 'TRABBXU128F92FEF48.json',
 'TRABBBV128F42967D7.json',
 'TRABBLU128F93349CF.json',
 'TRABBOP128F931B50D.json',
 'TRABBZN12903CD9297.json',
 'TRABCRU128F423F449.json',
 'TRABCTK128F934B224.json',
 'TRABCAJ12903CDFCC2.json',
 'TRABCPZ128F4275C32.json',
 'TRABCFL128F149BB0D.json',
 'TRABCUQ128E0783E2B.json',
 'TRABCEI128F424C983.json',
 'TRABCXB128F4286BD3.json',
 'TRABCEC128F426456E.json',
 'TRABCIX128F4265903.json',
 'TRABCKL128F423A778.json',
 'TRABCYE128F934CE1D.json',
 'TRABAZH128F930419A.json',
 'TRABAIO128F42938F9.json',
 'TRABAFP128F931E9A1.json',
 'TRABAWW128F4250A31.json',
 'TRABAXR128F426515F.json',
 'TRABAVQ12903CBF7E0.json',
 'TRABATO128F42627E9.json',
 'TRABACN128F425B784.json',
 'TRABAXL128F424FC50.json',
 'TRABAFJ128F42AF24E.json',
 'TRABAXV128F92F6AE3.json',
 'TRAABYW128F4244559

In [267]:
filepath = song_files[1]

In [268]:
df = pd.read_json(filepath, lines=True)
df.head()

,num_songs,artist_id,artist_latitude,artist_longitude,artist_location,artist_name,song_id,title,duration,year
0,1,ARDR4AC1187FB371A1,NaN,NaN,,Montserrat Caballé;Placido Domingo;Vicente Sar...,SOBAYLL12A8C138AF9,Sono andati? Fingevo di dormire,511.16363,0


## #1: `songs` Table
#### Extract Data for Songs Table
- Select columns for song ID, title, artist ID, year, and duration
- Use `df.values` to select just the values from the dataframe
- Index to select the first (only) record in the dataframe
- Convert the array to a list and set it to `song_data`

In [269]:
song_data = df[['song_id', 'title', 'artist_id', 'year', 'duration']].values[0]
song_data

array(['SOBAYLL12A8C138AF9', 'Sono andati? Fingevo di dormire',
       'ARDR4AC1187FB371A1', 0, 511.16363], dtype=object)

#### Insert Record into Song Table
Implement the `song_table_insert` query in `sql_queries.py` and run the cell below to insert a record for this song into the `songs` table. Remember to run `create_tables.py` before running the cell below to ensure you've created/resetted the `songs` table in the sparkify database.

In [270]:
cur.execute(song_table_insert, song_data)
conn.commit()

Run `test.ipynb` to see if you've successfully added a record to this table.

### Mini Example: Extracting Data For multiple songs

In [271]:
# It's better to make a list of dataframes then concat 
# rather than to append to a DataFrame directly
multiple_song_list = []
for f in song_files[0:20]:
    _df = pd.read_json(f, lines=True)
    multiple_song_list.append(_df)

j_df = pd.concat(multiple_song_list, ignore_index=True)
j_df.head()

,num_songs,artist_id,artist_latitude,artist_longitude,artist_location,artist_name,song_id,title,duration,year
0,1,AR36F9J1187FB406F1,56.27609,9.51695,Denmark,Bombay Rockers,SOBKWDJ12A8C13B2F3,Wild Rose (Back 2 Basics Mix),230.71302,0
1,1,ARDR4AC1187FB371A1,NaN,NaN,,Montserrat Caballé;Placido Domingo;Vicente Sar...,SOBAYLL12A8C138AF9,Sono andati? Fingevo di dormire,511.16363,0
2,1,AR62SOJ1187FB47BB5,NaN,NaN,,Chase & Status,SOGVQGJ12AB017F169,Ten Tonne,337.68444,2005
3,1,ARBGXIG122988F409D,37.77916,-122.42005,California - SF,Steel Rain,SOOJPRH12A8C141995,Loaded Like A Gun,173.19138,0
4,1,ARGCY1Y1187B9A4FA5,36.16778,-86.77836,"Nashville, TN.",Gloriana,SOQOTLQ12AB01868D0,Clementina Santafè,153.33832,0


## #2: `artists` Table
#### Extract Data for Artists Table
- Select columns for artist ID, name, location, latitude, and longitude
- Use `df.values` to select just the values from the dataframe
- Index to select the first (only) record in the dataframe
- Convert the array to a list and set it to `artist_data`

In [272]:
import numpy as np

In [273]:
df

,num_songs,artist_id,artist_latitude,artist_longitude,artist_location,artist_name,song_id,title,duration,year
0,1,ARDR4AC1187FB371A1,NaN,NaN,,Montserrat Caballé;Placido Domingo;Vicente Sar...,SOBAYLL12A8C138AF9,Sono andati? Fingevo di dormire,511.16363,0


### Handle Null Values for Postgres

In [274]:
df.isnull()

,num_songs,artist_id,artist_latitude,artist_longitude,artist_location,artist_name,song_id,title,duration,year
0,False,False,True,True,False,False,False,False,False,False


In [275]:
df=df.fillna(np.nan).replace(np.nan, None)
df

In [260]:
artist_data = df[['artist_id', 'artist_name', 'artist_location', 'artist_latitude', 'artist_longitude']].values[0]
artist_data

array(['ARDR4AC1187FB371A1',
       'Montserrat Caballé;Placido Domingo;Vicente Sardinero;Judith Blegen;Sherrill Milnes;Georg Solti',
       '', None, None], dtype=object)

#### Insert Record into Artist Table
Implement the `artist_table_insert` query in `sql_queries.py` and run the cell below to insert a record for this song's artist into the `artists` table. Remember to run `create_tables.py` before running the cell below to ensure you've created/resetted the `artists` table in the sparkify database.

In [261]:
cur.execute(artist_table_insert, artist_data)
conn.commit()

Run `test.ipynb` to see if you've successfully added a record to this table.

# Process `log_data`
In this part, you'll perform ETL on the second dataset, `log_data`, to create the `time` and `users` dimensional tables, as well as the `songplays` fact table.

Let's perform ETL on a single log file and load a single record into each table.
- Use the `get_files` function provided above to get a list of all log JSON files in `data/log_data`
- Select the first log file in this list
- Read the log file and view the data

In [18]:
log_files = get_files('data/log_data/2018/11/')

In [19]:
filepath = log_files[0]

In [20]:
df = pd.read_json(filepath, lines=True)
df.head()

,artist,auth,firstName,gender,itemInSession,lastName,length,level,location,method,page,registration,sessionId,song,status,ts,userAgent,userId
0,A Fine Frenzy,Logged In,Anabelle,F,0,Simpson,267.91138,free,"Philadelphia-Camden-Wilmington, PA-NJ-DE-MD",PUT,NextSong,1.541044e+12,256,Almost Lover (Album Version),200,1541377992796,"""Mozilla/5.0 (Macintosh; Intel Mac OS X 10_9_4...",69
1,Nirvana,Logged In,Aleena,F,0,Kirby,214.77832,paid,"Waterloo-Cedar Falls, IA",PUT,NextSong,1.541023e+12,237,Serve The Servants,200,1541381242796,Mozilla/5.0 (Macintosh; Intel Mac OS X 10.9; r...,44
2,Television,Logged In,Aleena,F,1,Kirby,238.49751,paid,"Waterloo-Cedar Falls, IA",PUT,NextSong,1.541023e+12,237,See No Evil (Remastered LP Version),200,1541381456796,Mozilla/5.0 (Macintosh; Intel Mac OS X 10.9; r...,44
3,JOHN COLTRANE,Logged In,Aleena,F,2,Kirby,346.43546,paid,"Waterloo-Cedar Falls, IA",PUT,NextSong,1.541023e+12,237,Blues To Bechet (LP Version),200,1541381694796,Mozilla/5.0 (Macintosh; Intel Mac OS X 10.9; r...,44
4,NOFX,Logged In,Aleena,F,3,Kirby,80.79628,paid,"Waterloo-Cedar Falls, IA",PUT,NextSong,1.541023e+12,237,It's My Job To Keep Punk Rock Elite,200,1541382040796,Mozilla/5.0 (Macintosh; Intel Mac OS X 10.9; r...,44


## #3: `time` Table
#### Extract Data for Time Table
- Filter records by `NextSong` action
- Convert the `ts` timestamp column to datetime
  - Hint: the current timestamp is in milliseconds
- Extract the timestamp, hour, day, week of year, month, year, and weekday from the `ts` column and set `time_data` to a list containing these values in order
  - Hint: use pandas' [`dt` attribute](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.Series.dt.html) to access easily datetimelike properties.
- Specify labels for these columns and set to `column_labels`
- Create a dataframe, `time_df,` containing the time data for this file by combining `column_labels` and `time_data` into a dictionary and converting this into a dataframe

In [21]:
df = df[df['page']=='NextSong'].copy()
df.head()

,artist,auth,firstName,gender,itemInSession,lastName,length,level,location,method,page,registration,sessionId,song,status,ts,userAgent,userId
0,A Fine Frenzy,Logged In,Anabelle,F,0,Simpson,267.91138,free,"Philadelphia-Camden-Wilmington, PA-NJ-DE-MD",PUT,NextSong,1.541044e+12,256,Almost Lover (Album Version),200,1541377992796,"""Mozilla/5.0 (Macintosh; Intel Mac OS X 10_9_4...",69
1,Nirvana,Logged In,Aleena,F,0,Kirby,214.77832,paid,"Waterloo-Cedar Falls, IA",PUT,NextSong,1.541023e+12,237,Serve The Servants,200,1541381242796,Mozilla/5.0 (Macintosh; Intel Mac OS X 10.9; r...,44
2,Television,Logged In,Aleena,F,1,Kirby,238.49751,paid,"Waterloo-Cedar Falls, IA",PUT,NextSong,1.541023e+12,237,See No Evil (Remastered LP Version),200,1541381456796,Mozilla/5.0 (Macintosh; Intel Mac OS X 10.9; r...,44
3,JOHN COLTRANE,Logged In,Aleena,F,2,Kirby,346.43546,paid,"Waterloo-Cedar Falls, IA",PUT,NextSong,1.541023e+12,237,Blues To Bechet (LP Version),200,1541381694796,Mozilla/5.0 (Macintosh; Intel Mac OS X 10.9; r...,44
4,NOFX,Logged In,Aleena,F,3,Kirby,80.79628,paid,"Waterloo-Cedar Falls, IA",PUT,NextSong,1.541023e+12,237,It's My Job To Keep Punk Rock Elite,200,1541382040796,Mozilla/5.0 (Macintosh; Intel Mac OS X 10.9; r...,44


In [22]:
t = pd.DataFrame(df['ts'].apply(lambda x: pd.to_datetime(x)))
t

,ts
0,1970-01-01 00:25:41.377992796
1,1970-01-01 00:25:41.381242796
2,1970-01-01 00:25:41.381456796
3,1970-01-01 00:25:41.381694796
4,1970-01-01 00:25:41.382040796
...,...
402,1970-01-01 00:25:41.451499796
403,1970-01-01 00:25:41.451636796
404,1970-01-01 00:25:41.453165796
405,1970-01-01 00:25:41.453431796


# Multiple Ways to create the time_df

# 1. Enumerate Loop then Add to df

In [29]:
time_data = [t.ts.dt.hour, t.ts.dt.day, t.ts.dt.weekofyear, 
             t.ts.dt.month, t.ts.dt.year, t.ts.dt.weekday]
column_labels = ['hour', 'day', 'week_of_year', 'month', 'year', 'weekday']

/tmp/ipykernel_15264/4003200255.py:1: FutureWarning: Series.dt.weekofyear and Series.dt.week have been deprecated. Please use Series.dt.isocalendar().week instead.
  time_data = [t.ts.dt.hour, t.ts.dt.day, t.ts.dt.weekofyear,


In [43]:
# add the time_data and column_labels to our time dataframe
for i, col in enumerate(time_data):
    t[column_labels[i]] = col

In [44]:
t.head()

,ts,hour,day,week_of_year,month,year,weekday,timestamp
0,1970-01-01 00:25:41.377992796,0,1,1,1,1970,3,1970-01-01 00:25:41.377992796
1,1970-01-01 00:25:41.381242796,0,1,1,1,1970,3,1970-01-01 00:25:41.381242796
2,1970-01-01 00:25:41.381456796,0,1,1,1,1970,3,1970-01-01 00:25:41.381456796
3,1970-01-01 00:25:41.381694796,0,1,1,1,1970,3,1970-01-01 00:25:41.381694796
4,1970-01-01 00:25:41.382040796,0,1,1,1,1970,3,1970-01-01 00:25:41.382040796


# 2. Convert to Object, Transpose, Zip Dict

In [41]:
# could be useful if needing all data as certain type
time_data = t.astype(object).values.T
time_df = pd.DataFrame(dict(zip(column_labels, time_data)))

In [42]:
time_df.head()

,timestamp,hour,day,week_of_year,month,year,weekday
0,1970-01-01 00:25:41.377992796,0,1,1,1,1970,3
1,1970-01-01 00:25:41.381242796,0,1,1,1,1970,3
2,1970-01-01 00:25:41.381456796,0,1,1,1,1970,3
3,1970-01-01 00:25:41.381694796,0,1,1,1,1970,3
4,1970-01-01 00:25:41.382040796,0,1,1,1,1970,3


# 3. Concat

In [36]:
time_data = [t.ts, t.ts.dt.hour, t.ts.dt.day, t.ts.dt.weekofyear, 
             t.ts.dt.month, t.ts.dt.year, t.ts.dt.weekday]
column_labels = ['timestamp', 'hour', 'day', 'week_of_year', 'month', 'year', 'weekday']
time_df = pd.concat(time_data, axis=1)
time_df.columns = column_labels
# https://www.w3resource.com/python-exercises/pandas/python-pandas-data-frame-exercise-72.php

/tmp/ipykernel_15264/3380099409.py:1: FutureWarning: Series.dt.weekofyear and Series.dt.week have been deprecated. Please use Series.dt.isocalendar().week instead.
  time_data = [t.ts, t.ts.dt.hour, t.ts.dt.day, t.ts.dt.weekofyear,


In [45]:
time_df.head()

,timestamp,hour,day,week_of_year,month,year,weekday
0,1970-01-01 00:25:41.377992796,0,1,1,1,1970,3
1,1970-01-01 00:25:41.381242796,0,1,1,1,1970,3
2,1970-01-01 00:25:41.381456796,0,1,1,1,1970,3
3,1970-01-01 00:25:41.381694796,0,1,1,1,1970,3
4,1970-01-01 00:25:41.382040796,0,1,1,1,1970,3


# 4. Directly Zip Dict

In [39]:
time_data = [t.ts, t.ts.dt.hour, t.ts.dt.day, t.ts.dt.weekofyear, 
             t.ts.dt.month, t.ts.dt.year, t.ts.dt.weekday]
column_labels = ['timestamp', 'hour', 'day', 'week_of_year', 'month', 'year', 'weekday']
time_df = pd.DataFrame(dict(zip(column_labels, time_data)))

/tmp/ipykernel_15264/1681114894.py:1: FutureWarning: Series.dt.weekofyear and Series.dt.week have been deprecated. Please use Series.dt.isocalendar().week instead.
  time_data = [t.ts, t.ts.dt.hour, t.ts.dt.day, t.ts.dt.weekofyear,


In [63]:
time_df.head()

,timestamp,hour,day,week_of_year,month,year,weekday
0,1970-01-01 00:25:41.377992796,0,1,1,1,1970,3
1,1970-01-01 00:25:41.381242796,0,1,1,1,1970,3
2,1970-01-01 00:25:41.381456796,0,1,1,1,1970,3
3,1970-01-01 00:25:41.381694796,0,1,1,1,1970,3
4,1970-01-01 00:25:41.382040796,0,1,1,1,1970,3
...,...,...,...,...,...,...,...
402,1970-01-01 00:25:41.451499796,0,1,1,1,1970,3
403,1970-01-01 00:25:41.451636796,0,1,1,1,1970,3
404,1970-01-01 00:25:41.453165796,0,1,1,1,1970,3
405,1970-01-01 00:25:41.453431796,0,1,1,1,1970,3


# Continuing On...

In [64]:
time_df.dtypes

timestamp       datetime64[ns]
hour                     int64
day                      int64
week_of_year             int64
month                    int64
year                     int64
weekday                  int64
dtype: object

In [66]:
# Get a sample of what we're sending to postgres
for i, row in time_df.iterrows():
    print(list(row))
    break

[Timestamp('1970-01-01 00:25:41.377992796'), 0, 1, 1, 1, 1970, 3]


#### Insert Records into Time Table
Implement the `time_table_insert` query in `sql_queries.py` and run the cell below to insert records for the timestamps in this log file into the `time` table. Remember to run `create_tables.py` before running the cell below to ensure you've created/resetted the `time` table in the sparkify database.

In [67]:
for i, row in time_df.iterrows():
    cur.execute(time_table_insert, list(row))
    conn.commit()

Run `test.ipynb` to see if you've successfully added records to this table.

## #4: `users` Table
#### Extract Data for Users Table
- Select columns for user ID, first name, last name, gender and level and set to `user_df`

In [69]:
list(df.columns)

['artist',
 'auth',
 'firstName',
 'gender',
 'itemInSession',
 'lastName',
 'length',
 'level',
 'location',
 'method',
 'page',
 'registration',
 'sessionId',
 'song',
 'status',
 'ts',
 'userAgent',
 'userId']

In [70]:
user_df = df[['userId', 'firstName', 'lastName', 'gender', 'level']].copy()

In [72]:
user_df.head()

,userId,firstName,lastName,gender,level
0,69,Anabelle,Simpson,F,free
1,44,Aleena,Kirby,F,paid
2,44,Aleena,Kirby,F,paid
3,44,Aleena,Kirby,F,paid
4,44,Aleena,Kirby,F,paid


#### Insert Records into Users Table
Implement the `user_table_insert` query in `sql_queries.py` and run the cell below to insert records for the users in this log file into the `users` table. Remember to run `create_tables.py` before running the cell below to ensure you've created/resetted the `users` table in the sparkify database.

In [73]:
for i, row in user_df.iterrows():
    cur.execute(user_table_insert, row)
    conn.commit()

Run `test.ipynb` to see if you've successfully added records to this table.

## #5: `songplays` Table
#### Extract Data and Songplays Table
This one is a little more complicated since information from the songs table, artists table, and original log file are all needed for the `songplays` table. Since the log file does not specify an ID for either the song or the artist, you'll need to get the song ID and artist ID by querying the songs and artists tables to find matches based on song title, artist name, and song duration time.
- Implement the `song_select` query in `sql_queries.py` to find the song ID and artist ID based on the title, artist name, and duration of a song.
- Select the timestamp, user ID, level, song ID, artist ID, session ID, location, and user agent and set to `songplay_data`

#### Insert Records into Songplays Table
- Implement the `songplay_table_insert` query and run the cell below to insert records for the songplay actions in this log file into the `songplays` table. Remember to run `create_tables.py` before running the cell below to ensure you've created/resetted the `songplays` table in the sparkify database.

In [85]:
df.head()

,artist,auth,firstName,gender,itemInSession,lastName,length,level,location,method,page,registration,sessionId,song,status,ts,userAgent,userId
0,A Fine Frenzy,Logged In,Anabelle,F,0,Simpson,267.91138,free,"Philadelphia-Camden-Wilmington, PA-NJ-DE-MD",PUT,NextSong,1.541044e+12,256,Almost Lover (Album Version),200,1541377992796,"""Mozilla/5.0 (Macintosh; Intel Mac OS X 10_9_4...",69
1,Nirvana,Logged In,Aleena,F,0,Kirby,214.77832,paid,"Waterloo-Cedar Falls, IA",PUT,NextSong,1.541023e+12,237,Serve The Servants,200,1541381242796,Mozilla/5.0 (Macintosh; Intel Mac OS X 10.9; r...,44
2,Television,Logged In,Aleena,F,1,Kirby,238.49751,paid,"Waterloo-Cedar Falls, IA",PUT,NextSong,1.541023e+12,237,See No Evil (Remastered LP Version),200,1541381456796,Mozilla/5.0 (Macintosh; Intel Mac OS X 10.9; r...,44
3,JOHN COLTRANE,Logged In,Aleena,F,2,Kirby,346.43546,paid,"Waterloo-Cedar Falls, IA",PUT,NextSong,1.541023e+12,237,Blues To Bechet (LP Version),200,1541381694796,Mozilla/5.0 (Macintosh; Intel Mac OS X 10.9; r...,44
4,NOFX,Logged In,Aleena,F,3,Kirby,80.79628,paid,"Waterloo-Cedar Falls, IA",PUT,NextSong,1.541023e+12,237,It's My Job To Keep Punk Rock Elite,200,1541382040796,Mozilla/5.0 (Macintosh; Intel Mac OS X 10.9; r...,44


In [76]:
# print a sample of what we're sending to postgres
for index, row in df.iterrows():
    print(row.artist, "|", row.song, row.length)
    break

A Fine Frenzy | Almost Lover (Album Version) 267.91138


In [77]:
for index, row in df.iterrows():

    # get songid and artistid from song and artist tables
    cur.execute(song_select, (row.song, row.artist, row.length))
    results = cur.fetchone()
    
    if results:
        songid, artistid, duration = results
    else:
        songid, artistid, duration = None, None, None

    # insert songplay record
    songplay_data = [index, row.ts, row.userId, row.level, songid, artistid, row.sessionId, row.location, row.userAgent]
    cur.execute(songplay_table_insert, songplay_data)
    conn.commit()

Run `test.ipynb` to see if you've successfully added records to this table.

# Close Connection to Sparkify Database

In [78]:
conn.close()

# Implement `etl.py`
Use what you've completed in this notebook to implement `etl.py`.